In [7]:
# columns:
#  'FGM', 'FGA','FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB','DREB',
#     'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS'


import pandas as pd
import numpy as np
from dateutil import parser

def calculate_ranks(punt=None,start_date=None , end_date=None):
    nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
           'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
           'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
           'WAS', 'ORL', 'GSW', 'NOP', 
           'MIA', 
           'DET']
    nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
           1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
           1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
           1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
           1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
           1610612753, 1610612747, 1610612758, 1610612748, 1610612765]

   



    nba_teams_df = pd.DataFrame(nba_teams)
    dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')


    df = pd.read_csv('large_boxscoretraditionalv2_df.csv')
    df=df[df['TEAM_ID'].isin(nba_team_ids)]
    df.drop_duplicates(subset = ['GAME_ID','PLAYER_ID'],inplace=True)
    df.dropna(subset=['MIN'],inplace=True)

#     print(df.columns)
    df.dropna(subset=['MIN'],inplace=True)
    
    
    df_g = pd.read_csv('all_games.csv')
    df_g=df_g[['GAME_ID','TEAM_ID','GAME_DATE']].drop_duplicates().dropna()
    df_g['GAME_DATE'] = pd.to_datetime(df_g['GAME_DATE'])
    df = df.merge(df_g, 
         how='inner',
         left_on=['GAME_ID','TEAM_ID'],
         right_on=['GAME_ID','TEAM_ID'])
    
    
    if start_date is not None:
        print('len before filter start date', len(df))
        print (df['GAME_DATE'].max())
        df=df[df['GAME_DATE']>=parser.parse(start_date)]
        print('len after filter  start date', len(df))
    if end_date is not None:
        print('len before filter  end date', len(df))
        df=df[df['GAME_DATE']<=end_date]
        print('len after filter  end date', len(df))
        
    MINIMUM_NUM_OF_GAMES = 3

    means = df.groupby(['PLAYER_ID','PLAYER_NAME']).filter(lambda x: len(x) > MINIMUM_NUM_OF_GAMES).groupby(['PLAYER_ID','PLAYER_NAME']).mean()
    means.drop(columns = ['GAME_ID','TEAM_ID','FG_PCT','FT_PCT','OREB','DREB','PF','PLUS_MINUS','FG3_PCT'], inplace=True)
    means['FG_PCT'] = means['FGM'] / means['FGA']
    means['FT_PCT'] = means['FTM'] / means['FTA']
    means['TO'] = means['TO']*-1
    means.dropna(inplace=True)
    means.drop(columns =['FGM','FTM','FGA','FTA','FG3A'],inplace=True)
    means.drop(columns =punt,inplace=True)
    
    sums = df.groupby(['PLAYER_ID','PLAYER_NAME']).filter(lambda x: len(x) > MINIMUM_NUM_OF_GAMES).groupby(['PLAYER_ID','PLAYER_NAME']).sum()
    sums.drop(columns = ['GAME_ID','TEAM_ID','FG_PCT','FT_PCT','OREB','DREB','PF','PLUS_MINUS','FG3_PCT'], inplace=True)
    sums['FG_PCT'] = sums['FGM'] / sums['FGA']
    sums['FT_PCT'] = sums['FTM'] / sums['FTA']
    sums['TO'] = sums['TO']*-1
    sums.dropna(inplace=True)
    sums.drop(columns =['FGM','FTM','FGA','FTA','FG3A'],inplace=True)
    sums.drop(columns =punt,inplace=True)

    # counter = df.groupby()
    checkup = means.rank(method='min', ascending =True, pct =True).merge(
                                sums.rank(method='min', ascending =True, pct =True),
                                how='inner',
                                left_index=True,
                                right_index=True,
    suffixes=['_mean','_sum'])
    checkup_rank = checkup.mean(axis=1).sort_values(ascending=False).to_frame(name='valued_rank')
    checkup_rank['rank_not_adj'] = checkup_rank.reset_index().index
    checkup_rank_with_size = checkup_rank.merge(np.log(df.groupby(['PLAYER_ID','PLAYER_NAME']).size()).to_frame(name='sample_size') ,
                      how='inner',
                      left_index=True,
                      right_index=True)
    checkup_rank_with_size['valued_rank_adj_to_games'] = (checkup_rank_with_size['valued_rank'] * (checkup_rank_with_size['sample_size'])).sort_values(ascending=False)
    checkup_rank_with_size = checkup_rank_with_size.sort_values('valued_rank_adj_to_games',ascending=False)
    checkup_rank_with_size['rank_adj'] = checkup_rank_with_size.reset_index().index
    return(checkup_rank_with_size)

ranks = calculate_ranks(punt=['FG3M','AST'])
# ranks = calculate_ranks(punt=['FG_PCT', 'FG3M', 'PTS'])
# ranks = calculate_ranks(punt=['FG3M','FG_PCT','FT_PCT','STL','BLK','TO'])

ranks = ranks.reset_index()
ranks.sort_values('rank_adj')[0:50]['PLAYER_NAME']


0                Nikola Jokic
1               Kawhi Leonard
2                James Harden
3           LaMarcus Aldridge
4               Tobias Harris
5               Pascal Siakam
6                 Brook Lopez
7              Nikola Vucevic
8               Anthony Davis
9          Karl-Anthony Towns
10               Jayson Tatum
11                 Al Horford
12              DeMar DeRozan
13      Giannis Antetokounmpo
14               Jimmy Butler
15                Danny Green
16                Jeremy Lamb
17             Damian Lillard
18             Richaun Holmes
19    Shai Gilgeous-Alexander
20               Bradley Beal
21                Rudy Gobert
22                Joel Embiid
23                Bam Adebayo
24               Paul Millsap
25               Kevin Durant
26          Mitchell Robinson
27               Nerlens Noel
28           Montrezl Harrell
29            Nemanja Bjelica
30              Dwight Powell
31             Andre Drummond
32                   Rudy Gay
33        

In [6]:
# compare teams
yair = 'Jru|Smart|Harde|Pasch|Nance|House|Capel|Jonas|Dejo|Wend|Covingt|Josh Ri|Langs|Lamb'
nachi = 'Sato|Jru|Hach|Thomas Br|Millsa|Brog|Tobi|Vuce|Smart|Tim Ha|Wende|Beve|Noel|te Murray|OG'
oren = 'Zach|Moore|Barton|Kuzma|George|Griffi|Aldri|Jarret|Oubre|Nunn|Gasol|Irving|Dragic|Wiggins|Fox'
josh = 'Morant|Brooks|Bridge|Powell|Collins|Davis|Ayton|Rozier|Hood|Wright|Dunn|Herro|Porter'
yaron = 'Booker|Harris|Shai|Jae C|OG|PJ|Richau|Derrick F|George H|Rondo|Tatum|Turner|Mutchell R'
yoni='CJ|Teague|Barret|Kaminsky|Markkan|Tristan|Whiteside|Schro|Aaron Hol|Evan|Aaron Gordon|Joel|Ingram'
david="Collin Sexton|Bradley Beal|Buddy Hield|Joe Harris|Brandon Clarke|Kenrich Williams|Montrezl Harrell|Brook Lopez|Devonte' Graha|Nerlens Noel|Cody Zeller|Pascal Siakam|Jonathan Isaac|Kawhi Leonard|Serge Ibaka"
Or = 'Coby W|n Clarks|Booker|Glen Rob|Bojan Bog|Danilo G|Holmes|Saric|Redick|Rubio|Jokic|Porzin|Dinw'
ori = 'Paul|Butler|Lou |Warren|LeBron James|Dwight|dre Jordan|Al Ho|Burks|DeRo| Rose|Gay'

ranks[ranks['PLAYER_NAME'].str.contains(ori + nachi)].sort_values('rank_not_adj')


,PLAYER_ID,PLAYER_NAME,valued_rank,rank_not_adj,sample_size,valued_rank_adj_to_games,rank_adj
4,203457,Nerlens Noel,0.737361,7,3.135494,2.311992,4
8,200794,Paul Millsap,0.736404,8,3.091042,2.276255,8
5,202699,Tobias Harris,0.715052,19,3.218876,2.301663,5
26,201143,Al Horford,0.704328,27,3.135494,2.208416,26
134,202696,Nikola Vucevic,0.698774,36,2.639057,1.844106,134
50,202710,Jimmy Butler,0.697913,37,2.995732,2.090760,50
10,1628976,Wendell Carter Jr.,0.695615,40,3.258097,2.266380,10
30,201942,DeMar DeRozan,0.693221,44,3.135494,2.173590,30
32,1628384,OG Anunoby,0.691210,46,3.135494,2.167286,32
41,1629060,Rui Hachimura,0.680774,55,3.135494,2.134562,41


In [72]:
yaron_pro = 'Batum|Mathews|Hollis|y Grant|s Young|ny Davis|Hield|n Lopez|Dotson|Kanter|Gallina|i Parker '
nachi_pro = 'Nti|Beal|McCollum|Osman|Siakam|Portis|Redick|Kelly Ou|Wanamak|Dunn|Kentav|Lou Will'
nachi = 'Sato|Jru|Hach|Thomas Br|Millsa|Brog|Tobi|Vuce|Smart|Tim Ha|Wende|Beve|Noel|te Murray|Duncan Rob'
ori = 'Paul|Butler|Lou |Warren|LeBron James|Dwight|dre Jordan|Al Ho|Burks|DeRo| Rose|Gay'
# ori = oren

ranks = calculate_ranks(punt=['FT_PCT','FG_PCT'])
ranks=ranks.reset_index()
first_ranks = ranks[ranks['PLAYER_NAME'].str.contains( ori + '|' + nachi)].sort_values('rank_not_adj')

ranks = calculate_ranks(punt=['FG3M' ,'PTS'])
ranks=ranks.reset_index()
second_ranks = ranks[ranks['PLAYER_NAME'].str.contains(nachi+ '|' + ori)].sort_values('rank_not_adj')

tmp = first_ranks.merge(second_ranks, how='inner', left_on='PLAYER_NAME' , right_on='PLAYER_NAME', suffixes = ['_For_yaron' , '_For_Nachi'])
tmp[['PLAYER_NAME','rank_not_adj_For_yaron','rank_not_adj_For_Nachi']].sort_values('rank_not_adj_For_Nachi')


,PLAYER_NAME,rank_not_adj_For_yaron,rank_not_adj_For_Nachi
24,Nerlens Noel,224,3
0,Al Horford,3,12
4,Tobias Harris,11,18
3,Jimmy Butler,10,20
9,Paul Millsap,65,25
8,Nikola Vucevic,39,26
14,DeMar DeRozan,107,28
1,LeBron James,6,31
26,DeAndre Jordan,272,41
15,Thomas Bryant,110,42


In [4]:
# check punt

# spec_team = 'Sato|Bj|Hachi|Thomas Br|Millsa|Caul|Brog|Tobi|Vuce|Ntl|ue Holiday|Wendel|te Murray|Smart'
spec_team = 'Jrue Holiday|Dillon Brooks|Dennis Schroder|Jae Crowder|Jabari Parker|Aaron Gordon|Thomas Bryant|Luka Doncic|Blake Griffin|Jarrett Allen|Willie Cauley|James Harden|Bogdan Bogdanovic'
yaron_pro = 'Batum|Mathews|Hollis|y Grant|s Young|ny Davis|Hield|n Lopez|Dotson|Kanter|Gallina|i Parker '
nachi_pro = 'Beal|McCollum|Osman|Siakam|Portis|Redick|Kelly Ou|Bazem|Dunn|Kentav|Lou Will|Seth Cu' # REB and FG
nachi_ballerz     ='Sato|Millsa|Brog|Wendel|te Murray|Noel|Harrel|a Vuc|Clarke|Derrick Jones|Caldw|Zeller|Bjel' #
nachi_platinum = 'Jrue H|on Brooks|Harden|Donci|Crowder|ond Green|ett Allen|Blake Gri|Bogdan Bo|Schroder|Aaron Gordon|Thomas Bryant|DiVi|Pasch'
Or = 'Coby W|n Clarks|Booker|Glen Rob|Bojan Bog|Danilo G|Holmes|Saric|Redick|Rubio|Jokic|Porzin|Dinw'


nachi_alt='Sato|Millsa|Brog|Wendel|te Murray|Noel|Harrel|a Vuc|Clarke|Derrick Jones|Caldw|Zeller|Bjel' #

curr_team = 'Anthony Davis|Nerlens Noel|Brandon Clarke|Steven Adams|Robert Covington|Brook Lopez|Miles Bridges|Pascal Siakam|Jeremy Lamb|T.J. Warren|Patrick Beverley|Kris Dunn|Mike Conley'

import itertools
list_of_categories = ['FG_PCT', 'FG3M',  'FT_PCT', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PTS']
list_of_combinations = []
length_of_combinations = [0,1,2,3]

for len_of_com in length_of_combinations:
    combs = itertools.combinations(list_of_categories, len_of_com)
    for subset in combs:
            list_of_combinations.append(subset)

dic_of_ranks_by_punt  = {}            
for comb in list_of_combinations:
    ranks = calculate_ranks(punt=list(comb)).reset_index()
    sum_rank = ranks[ranks['PLAYER_NAME'].str.contains(curr_team)]['rank_not_adj'].sum()
    dic_of_ranks_by_punt[comb] = sum_rank
pd.DataFrame.from_dict(dic_of_ranks_by_punt, orient = 'index', columns=['summed_rank']).sort_values('summed_rank')

,summed_rank
"(FG_PCT, FG3M, FT_PCT)",1152
"(FG_PCT, FG3M)",1223
"(FG_PCT, FG3M, PTS)",1232
"(FG_PCT, FG3M, STL)",1291
"(FG3M, PTS)",1322
"(FG_PCT, FG3M, AST)",1333
"(FG3M,)",1338
"(FT_PCT, PTS)",1349
"(FG3M, FT_PCT)",1349
"(FG_PCT, FG3M, REB)",1352


In [14]:
# nachi = 'Sato|Hachi|Thomas Br|Millsa|Brog|Vuce|Wendel|te Murray|Bev|Noel|Siak|Harrel|Clarke|ge Hill|Baynes|Lamb|Coving'
nachi='Jones |Bazemore|h Thomas|Price|Ilya|ge Hill|en Robinson|Hach|Zeller'
# nachi_pro = 'Nti|Beal|McCollum|Osman|Siakam|Portis|Redick|Kelly Ou|Wanamak|Dunn|Kentav|Lou Will|Mills|White|Baze'
# nachi_platinum = 'Jrue H|on Brooks|Harden|Donci|Crowder|ond Green|ett Allen|Blake Gri|Bogdan Bo|Schroder|Aaron Gordon|Thomas Bryant|DiVi|Hill|Hunter'

spec_team = 'Kleb|Finn|Jones J|White|Porter'



ranks = calculate_ranks(punt=['FG3M','AST','PTS'], start_date='2020-01-10')

# ranks = calculate_ranks(punt=['FG_PCT','TO'], start_date='2019-12-10')
ranks= ranks.reset_index()

ranks[ranks['PLAYER_NAME'].str.contains(spec_team)].sort_values('rank_adj')[['PLAYER_NAME','rank_not_adj','rank_adj']]

# ranks.sort_values('rank_adj').iloc[30:70][['PLAYER_NAME','rank_adj','rank_not_adj']]

len before filter start date 44717
2020-01-18 00:00:00
len after filter  start date 1494


,PLAYER_NAME,rank_not_adj,rank_adj
19,Michael Porter Jr.,4,19
31,Maxi Kleber,47,31
66,Dorian Finney-Smith,93,66
83,Derrick Jones Jr.,55,83
124,Hassan Whiteside,95,124
139,Derrick White,114,139
200,Coby White,226,200


In [82]:
# compare biggest jumps with punt - without

ranks_a = calculate_ranks(punt=['FG3M','AST'])
ranks_b = calculate_ranks(punt=[])

merge_ranks = ranks_a[['rank_adj']].merge(ranks_b[['rank_adj']] ,
                                how='inner', 
                                left_index=True , 
                                right_index=True, 
                                suffixes = ['_a','_b'])

merge_ranks['diff_between_punts'] = merge_ranks['rank_adj_a'] - merge_ranks['rank_adj_b']
merge_ranks[(merge_ranks['rank_adj_a']<150)&
           (merge_ranks['rank_adj_b']<150)&
           (merge_ranks['diff_between_punts']<0)].sort_values('diff_between_punts')

,,rank_adj_a,rank_adj_b,diff_between_punts
PLAYER_ID,PLAYER_NAME,,,
1629011,Mitchell Robinson,8,142,-134
203497,Rudy Gobert,40,140,-100
1628386,Jarrett Allen,15,113,-98
202355,Hassan Whiteside,29,126,-97
1626161,Willie Cauley-Stein,20,114,-94
203457,Nerlens Noel,6,95,-89
1628976,Wendell Carter Jr.,18,100,-82
1626149,Montrezl Harrell,21,98,-77
1627737,Marquese Chriss,64,131,-67
